In [3]:
from transformers import T5ForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import RobertaTokenizer
from datasets import DatasetDict
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback
from datasets import load_dataset
import pandas as pd
import black
import os
import re
from datasets import Dataset
import autopep8

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# ------------------------
# 1. Install Required Libraries
# ------------------------
#!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124
#!pip install transformers datasets evaluate -q

In [8]:
#https://huggingface.co/Salesforce/codet5-small
# ------------------------------------------------------------------------
# 2. Load Dataset (CodeXGLUE - Code Translation Java <=> C#)
# ------------------------------------------------------------------------
data_dir = r"C:\Users\bentr\Downloads\Archive\Archive"

# CodeXGLUE is a benchmark dataset collection by Microsoft for code-related tasks.
# Here, we use the code-translation-python-java dataset.
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
# Read the CSV files into DataFrames
test_dataset = load_dataset('csv', data_files=os.path.join(data_dir, csv_files[0]))['train']
train_dataset = load_dataset('csv', data_files=os.path.join(data_dir, csv_files[1]))['train']
validation_dataset = load_dataset('csv', data_files=os.path.join(data_dir, csv_files[2]))['train']

dataset = DatasetDict({
    'test': test_dataset,
    'train': train_dataset,
    'validation': validation_dataset
})

In [9]:
model_checkpoint = "Salesforce/codet5-small"
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)
tokenizer.add_tokens(["<MASK>"]) #Imagine we need an extra token. This line adds the extra token to the vocabulary

model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(32101, 512)

In [ ]:
flattened_method = dataset["train"]["cleaned_method"][2]

# Flatten the method by joining words with a single space
flattened_method = " ".join(flattened_method.split())
flattened_method =re.sub(r'\s*([=+\-*/%<>!&|^(),:{}\[\].])\s*', r'\1', flattened_method)
# Get and flatte n the target block
target = dataset["train"]["target_block"][2]
target = re.sub(r'\s*([=+\-*/%<>!&|^(),:{}\[\].])\s*', r'\1', target)

print("Target after normalization:", target)

# Replace target with <MASK> in the flattened method
flattened_method = flattened_method.replace(target, "<MASK>")

# Output the result
print("Flattened method after replacemooent:", flattened_method)
print("Target:", target)  # Shows the target for reference
#".egg" + path . sep in filename :  #if ".egg" + path . sep in filename:

In [ ]:
"""def mask_dataset(dataset):
    # Create lists to store processed methods and targets
    processed_methods = []
    processed_targets = []
    i = 0

    # Loop through the dataset and apply processing
    yes = 0
    no = 0
    while i < 1000:
        # Get the current method and target block
        i += 1
        flattened_method = dataset["train"]["cleaned_method"][i]
        target = dataset["train"]["target_block"][i]

        # Flatten the method by joining words with a single space
        flattened_method = " ".join(flattened_method.split())
        flattened_method = re.sub(r'\s*([=+\-*/%<>!&|^(),:{}\[\].])\s*', r'\1', flattened_method)

        # Normalize the target block
        target = re.sub(r'\s*([=+\-*/%<>!&|^(),:{}\[\].])\s*', r'\1', target)

        # Replace target with <MASK> in the flattened method
        if target not in flattened_method:
            no+=1
        if target in flattened_method:
            flattened_method = flattened_method.replace(target, "<MASK>")
            yes+=1
        # Append processed results
        processed_methods.append(flattened_method)
        processed_targets.append(target)
    print(yes)
    print(no)
    # Build Dataset (not DatasetDict)
    processed = Dataset.from_dict({
        'processed_target': processed_targets,
        'processed_method': processed_methods,
    })
    return processed

# Example usage
# Assuming 'dataset' is your existing dataset variable
processed = mask_dataset(dataset)

# Check a sample processed method and target
print(processed["processed_method"][0])
print(processed["processed_target"][0])"""

In [ ]:
def mask_dataset(dataset):

    processed_methods = []
    processed_targets = []
    i = 0

    # Loop through the dataset and apply processing
    yes = 0
    no = 0
    while i <= 49999:
        # Get the current method and target block
            if (i + 1) % 250 == 0: print(f"Processed {i + 1}")
            flattened_method = dataset["train"]["cleaned_method"][i]
            target = dataset["train"]["target_block"][i]

        # Flatten the method by joining words with a single space
            flattened_method = " ".join(flattened_method.split())
            flattened_method = re.sub(r'\s*([=+\-*/%<>!&|^(),:{}\[\].])\s*', r'\1', flattened_method)

        # Normalize the target block
            target = re.sub(r'\s*([=+\-*/%<>!&|^(),:{}\[\].])\s*', r'\1', target)

        # Replace target with <MASK> in the flattened method
            if target not in flattened_method:
                no+=1
            if target in flattened_method:
                flattened_method = flattened_method.replace(target, "<MASK>")
                yes+=1
        # Append processed results
            processed_methods.append(flattened_method)
            processed_targets.append(target)
            i += 1
    print(yes)
    print(no)
    # Build Dataset (not DatasetDict)
    processed = Dataset.from_dict({
        'processed_target': processed_targets,
        'processed_method': processed_methods,
    })
    return processed
#variable for processed datatset
processed = mask_dataset(dataset)
#make sure dataset is masked properly 
print(processed["processed_method"][0])
print(processed["processed_target"][0])

In [4]:
# Convert to Pandas DataFrame
df = processed.to_pandas()

# Save to CSV
df.to_csv("masked_dataset.csv", index=False)

NameError: name 'processed' is not defined

In [6]:
processed = Dataset.from_pandas(pd.read_csv("masked_dataset.csv"))
print(processed)

Dataset({
    features: ['processed_target', 'processed_method'],
    num_rows: 50000
})


In [10]:
def preprocess_function(examples):
    inputs = examples["processed_method"]
    targets = examples["processed_target"]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=256, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



tokenized_datasets = processed.map(preprocess_function, batched=True)
print(tokenized_datasets)

Map: 100%|██████████| 50000/50000 [00:42<00:00, 1164.15 examples/s]

Dataset({
    features: ['processed_target', 'processed_method', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 50000
})


In [ ]:
print(tokenized_datasets["processed_method"])

In [ ]:
# ------------------------------------------------------------------------
# 5. Define Training Arguments and Trainer
# ------------------------------------------------------------------------


training_args = TrainingArguments(
    output_dir="./codet5-finetuned2",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_total_limit=2,
    logging_steps=100,
    push_to_hub=False,
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["processed_method"],
    eval_dataset=tokenized_datasets["processed_target"],
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

In [ ]:
# ------------------------
# 6. Train the Model
# ------------------------
trainer.train()

In [ ]:
# Evaluate 
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)
print("SacreBLEU Score: ", results)
language = "python"
print(predictions)
print(references)
# Compute CodeBLEU
#score = calc_codebleu(references, predictions, language)
#print("CodeBLEU Score: ", score)
res = calc_codebleu([[ref] for ref in references], predictions, lang="python")
print(res)
import numpy as np
"""
SacreBLEU Score:  {'bleu': 0.8265168183793802, 'precisions': [0.9166666666666666, 0.8181818181818182, 0.8, 0.7777777777777778], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 12, 'reference_length': 12}
['def sum(a, b): return a + b']
['def add(a, b): return a + b']
{'codebleu': 0.8251908791628888, 'ngram_match_score': 0.6434588841607617, 'weighted_ngram_match_score': 0.6573046324907937, 'syntax_match_score': 1.0, 'dataflow_match_score': 1.0}
Exact Match Score: 0.00
"""
len(predictions)
len(references)
exact_match_score = np.mean([ref == pred for ref, pred in zip(references, predictions)])
print(f"Exact Match Score: {exact_match_score:.2f}")